현재 환경에서 YOLOv8을 GPU로 추론할 수 있는지 확인

In [1]:
import torch
if torch.cuda.is_available():
    print('GPU 사용 가능')
else:
    print('GPU 사용 불가. CPU를 통한 추론만 가능')

GPU 사용 가능


기본 필요 패키지 import

In [8]:
import os
import cv2
from ultralytics import YOLO

테스트 이미지 로드해보기

In [10]:
# 이미지 경로 지정
image_path = './test images'

# 이미지 리스트 불러와보기
image_name_list = os.listdir(image_path)
image_name_list

['image_1.png', 'image_2.png']

In [35]:
# 이미지 표시해보기
img_list = []
for image_name in image_name_list:
    # cv2로 이미지 불러오기
    img = cv2.imread(f'{image_path}/{image_name}')
    # 비율을 해치지 않고 리사이즈 하는 과정
    h, w, c = img.shape
    img = cv2.resize(img, (640, int(h/w*640)))
    # 리스트에 이미지 저장(후추 YOLO 추론을 위함)
    img_list.append(img)
    # 이미지 표시하기
    cv2.imshow('test image show', img)
    cv2.waitKey(0) # 아무 키를 누르면 다음으로 넘어감
# 코드가 끝난 뒤 이미지 창 닫아주기
cv2.destroyAllWindows()

In [36]:
# 이미지가 어떻게 생겼는지 관찰하기
img_list[0]

array([[[182, 132,  43],
        [177, 127,  39],
        [175, 124,  37],
        ...,
        [104,  71,  44],
        [105,  71,  44],
        [105,  71,  44]],

       [[191, 140,  45],
        [189, 139,  46],
        [186, 136,  43],
        ...,
        [106,  72,  44],
        [106,  72,  44],
        [106,  72,  44]],

       [[198, 146,  49],
        [198, 146,  48],
        [197, 144,  46],
        ...,
        [107,  72,  43],
        [107,  72,  45],
        [107,  72,  45]],

       ...,

       [[134, 105,  55],
        [134, 104,  57],
        [134, 103,  56],
        ...,
        [193, 143,  49],
        [192, 143,  48],
        [192, 143,  47]],

       [[134, 104,  55],
        [134, 104,  55],
        [133, 104,  54],
        ...,
        [193, 144,  51],
        [193, 144,  50],
        [191, 142,  46]],

       [[133, 104,  54],
        [133, 104,  54],
        [133, 104,  54],
        ...,
        [193, 144,  51],
        [193, 144,  50],
        [193, 144,  49]]

YOLOv8로 이미지 추론해보기

In [37]:
# YOLO 모델 로드
model_dic = {'n':'yolov8n.pt', 
             's':'yolov8s.pt', 
             'm':'yolov8m.pt', 
             'l': 'yolov8l.pt', 
             'x': 'yolov8x.pt', 
             '1080x': 'yolo'}
model = YOLO(model_dic['x'])

In [38]:
# 이미지 넣기
test_img = img_list[0]
result = model.predict(source = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB), verbose = False)[0]
result

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant',

In [39]:
# pre-trained 모델의 class list 불러오기
class_list = result.names
class_list

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [40]:
# 우리가 원하는 형태로 후처리 하기(bbox와 class 이름)
results = result.boxes.data.tolist()
results

[[192.26681518554688,
  128.29116821289062,
  324.4732360839844,
  313.5286560058594,
  0.9254348874092102,
  0.0],
 [432.61962890625,
  83.29297637939453,
  548.6654052734375,
  273.46148681640625,
  0.896065354347229,
  0.0],
 [293.00469970703125,
  119.42230987548828,
  403.75701904296875,
  227.19317626953125,
  0.8083090782165527,
  0.0],
 [157.49020385742188,
  28.31103515625,
  196.29910278320312,
  85.83868408203125,
  0.7552218437194824,
  0.0],
 [10.61409854888916,
  175.37918090820312,
  39.07166290283203,
  210.255126953125,
  0.6687518358230591,
  0.0],
 [426.3678894042969,
  0.7651710510253906,
  450.2699890136719,
  55.111568450927734,
  0.5911059379577637,
  0.0],
 [128.76058959960938,
  181.66921997070312,
  176.75576782226562,
  241.11984252929688,
  0.5569432377815247,
  0.0],
 [69.22067260742188,
  160.48236083984375,
  115.5289306640625,
  243.58319091796875,
  0.4555675685405731,
  0.0],
 [178.72128295898438,
  74.23811340332031,
  196.29684448242188,
  86.4284210

In [41]:
# dic_list에 이쁘게 변환해서 넣어보기
dic_list = []
for result in results:
    x1, y1, x2, y2, conf, cls = int(result[0]), int(result[1]), int(result[2]), int(result[3]), round(float(result[4]), 2), int(result[5])
    class_name = class_list[cls] # class이름 추출
    dic_list.append({'bbox':[x1, y1, x2, y2], 'conf':conf, 'class_name':class_name})
dic_list

[{'bbox': [192, 128, 324, 313], 'conf': 0.93, 'class_name': 'person'},
 {'bbox': [432, 83, 548, 273], 'conf': 0.9, 'class_name': 'person'},
 {'bbox': [293, 119, 403, 227], 'conf': 0.81, 'class_name': 'person'},
 {'bbox': [157, 28, 196, 85], 'conf': 0.76, 'class_name': 'person'},
 {'bbox': [10, 175, 39, 210], 'conf': 0.67, 'class_name': 'person'},
 {'bbox': [426, 0, 450, 55], 'conf': 0.59, 'class_name': 'person'},
 {'bbox': [128, 181, 176, 241], 'conf': 0.56, 'class_name': 'person'},
 {'bbox': [69, 160, 115, 243], 'conf': 0.46, 'class_name': 'person'},
 {'bbox': [178, 74, 196, 86], 'conf': 0.31, 'class_name': 'skateboard'},
 {'bbox': [70, 160, 98, 190], 'conf': 0.31, 'class_name': 'person'},
 {'bbox': [587, 166, 625, 209], 'conf': 0.28, 'class_name': 'person'}]

In [48]:
# 직접 이미지 위에 그림을 그려보기
import copy
img = copy.deepcopy(test_img)
for dic in dic_list:
    cv2.rectangle(img, (dic['bbox'][0], dic['bbox'][1]), (dic['bbox'][2], dic['bbox'][3]), (0,0,255), 1)
    text = f'{dic["class_name"]}:{round(dic["conf"], 2)}'
    cv2.putText(img, text, (dic['bbox'][0], dic['bbox'][1]+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
cv2.imshow('draw image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

전 과정을 자동화 해보기 + 이미지 저장

In [54]:
output_path = './output'
os.makedirs(output_path, exist_ok=True)
for img_name in os.listdir(image_path):
    img = cv2.imread(f'{image_path}/{img_name}')
    h, w, c = img.shape
    img = cv2.resize(img, (640, int(h/w*640)))
    result = model.predict(source = cv2.cvtColor(img, cv2.COLOR_BGR2RGB), verbose = False)[0]
    results = result.boxes.data.tolist()
    dic_list = []
    for result in results:
        x1, y1, x2, y2, conf, cls = int(result[0]), int(result[1]), int(result[2]), int(result[3]), round(float(result[4]), 2), int(result[5])
        class_name = class_list[cls] # class이름 추출
        dic_list.append({'bbox':[x1, y1, x2, y2], 'conf':conf, 'class_name':class_name})
    for dic in dic_list:
        cv2.rectangle(img, (dic['bbox'][0], dic['bbox'][1]), (dic['bbox'][2], dic['bbox'][3]), (0,0,255), 1)
        text = f'{dic["class_name"]}:{round(dic["conf"], 2)}'
        cv2.putText(img, text, (dic['bbox'][0], dic['bbox'][1]+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
    # 이미지 저장
    cv2.imwrite(f'{output_path}/{img_name}', img)
    cv2.imshow('draw image', img)
    cv2.waitKey(0)
cv2.destroyAllWindows()

웹캠 환경에서 실시간 추론 가능

In [6]:
# 파이썬 기본 패키지
import threading
import time
from copy import deepcopy

# 추가 설치 패키지
from ultralytics import YOLO
import numpy as np
import cv2
import torch

'''
필요 기능:
- 입력: cv2_bgr_image, depth_map
- 출력: 감지 되었는지 return value, 사물 정보 dic_list: [{'class_name':person, 'conf':0.5, ''}]

함수 인수 정보
__init__.: YOLOv8 모델 버전
detect: conf_thresh=0.65, class_filter=None, narmalized=True
draw: 안에 있는 self.img랑 self.dic_list로 그린다
'''


class Custom_YOLOv8:
    def __init__(self, model_name='yolov8n.pt', filter=None):
        '''
        입력되는 Image와 Depth Map에서 사물인식하여, 감지된 사물의 결과의 x, y, depth 정보를 반환
        
        model_name : YOLOv8 model 이름 입력(yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt, 혹은 custom model 입력)
        filter : 감지를 원하는 특정 사물이 있을 경우 list형태로 입력. None일 경우 모두 추론(None or ['person', 'cat'])
        '''
        print('자세한 기능 사용법 안내: https://github.com/Nyan-SouthKorea/YOLOv8_for_ROS2')
        self._check_cuda()
        self.model = YOLO(model_name)
        self.filter = filter
        self.img = None
        self.depth_map = None
        self.dic_list = []
        self.run_request = False
        self.thread = threading.Thread(target = self._prediction_thread)
        self.thread.start()


    def run(self, img, depth_map, conf_thresh=0.65):
        '''
        추론하기 원하는 image와 depth_map을 입력.
        class 내부에서 multi-thread로 구동되는 YOLOv8 모델에 img와 depth_map을 업데이트하고 지금까지 처리된 가장 최근의 추론 결과를 반환.
        (이렇게 처리하는 이유는, 성능이 낮은 컴퓨터에서도 렉이 걸리지 않게끔 보이기 위해서임)
        
        img : cv2로 처리할 수 있는 numpy 배열의 bgr 이미지
        depth_map : numpy 배열의 2차원 1채널 거리 정보(안에 숫자들은 int, float 상관 없음)
        conf_thresh : confidence threshold 설정
        '''
        # 처리해야 하는 img, depth_map 업데이트
        self.img = img
        self.depth_map = depth_map
        self.conf_thresh = conf_thresh
        self.run_request = True
        
        # 가장 최근까지 처리된 추론 결과 반환
        if self.dic_list == []:
            return_value = False
        else:
            return_value = True
        return return_value, self.dic_list
    
    def draw(self):
        '''
        내부 변수 self.img와 self.dic_list를 활용하여 그려서 결과 반환
        '''
        img = deepcopy(self.img)
        dic_list = deepcopy(self.dic_list)
        for dic in self.dic_list:
            cv2.rectangle(img, (dic['bbox'][0], dic['bbox'][1]), (dic['bbox'][2], dic['bbox'][3]), (0,0,255), 2)
            text = f'{dic["class_name"]}:{round(dic["conf"], 2)}, depth: {dic["depth"]}'
            cv2.putText(img, text, (dic['bbox'][0], dic['bbox'][1]+25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        return img


    def _prediction_thread(self):
        '''
        run 함수에서 업데이트된 self.img와 self.depth_map을 multi-thread로 처리하여 추론 결과를 self.dic_list에 업데이트 함
        '''
        class_list = None
        while True:
            # run()에서 이미지를 투입할 때 마다 구동
            if self.run_request == False:
                time.sleep(0.01)
                continue
            self.run_request = False

            # run()에서 투입된 정보가 로직이 실행되는 중간에 섞이지 않도록 내부 함수로 업데이트
            img = deepcopy(self.img)
            depth_map = deepcopy(self.depth_map)
            conf_thresh = deepcopy(self.conf_thresh)

            # 추론
            results = self.model.predict(source = cv2.cvtColor(img, cv2.COLOR_BGR2RGB), verbose = False)[0]
            if class_list == None:
                class_list = results.names
            results = results.boxes.data.tolist()
            dic_list = []
            for result in results:
                x1, y1, x2, y2, conf, cls = int(result[0]), int(result[1]), int(result[2]), int(result[3]), round(float(result[4]), 2), int(result[5])
                class_name = class_list[cls] # class이름 추출
                # conf_thresh 넘는 경우만 계산
                if conf < conf_thresh:
                    continue

                # 만약 filter를 설정해 놓았을 경우, 원하는 class만 감지하기
                if self.filter != None:
                    if not class_name in self.filter:
                        continue

                # 감지된 사물의 x, y 포인트 계산
                center_x = int((x2+x1)/2)
                center_y = int((y2+y1)/2)
                # depth 계산
                depth = self._get_depth(depth_map, [x1,y1,x2,y2]) # depth 추출
                dic_list.append({'bbox':[x1, y1, x2, y2], 'conf':conf, 
                                 'class_name':class_name, 'center_x':center_x, 'center_y':center_y, 'depth':depth})
            # 실시간 추론값 업데이트
            self.dic_list = deepcopy(dic_list)

    def _get_depth(self, depth_map, bbox, rate=0.3):
        '''
        depth_map에서 bbox영역의 중앙 영역 depth를 반환

        depth_map : 내부 변수
        bbox : 내부 변수
        rate : bbox 면적에서 해당 rate만큼의 중앙 영역의 평균 값을 계산함
        '''
        # rate 반영한 bbox 계산
        x1, y1, x2, y2 = bbox
        # bbox의 가로 세로 길이
        x_len = x2-x1
        y_len = y2-y1
        # rate 반영한 bbox의 가로 세로 길이
        new_x_len = x_len*rate
        new_y_len = y_len*rate
        # rate 반영된 bbox 수치 계산
        new_x1 = x1 + ((x_len-new_x_len)/2)
        new_y1 = y1 + ((y_len-new_y_len)/2)
        new_x2 = x2 - ((x_len-new_x_len)/2)
        new_y2 = y2 - ((y_len-new_y_len)/2)
        new_x1, new_y1, new_x2, new_y2 = int(new_x1), int(new_y1), int(new_x2), int(new_y2)
        # depth_map에서 원하는 부분 crop하여 평균 depth 구하기
        crop_depth = depth_map[new_y1:new_y2, new_x1:new_x2]
        mean_depth = np.mean(crop_depth)
        return round(mean_depth, 2)
    
    def _check_cuda(self):
        '''
        현재 설치된 torch 환경이 YOLO를 GPU에서 구동할 수 있는 환경인지 검사
        '''
        if torch.cuda.is_available():
            print('GPU 사용 가능')
        else:
            print('GPU 사용 불가. CPU를 통한 추론만 가능')
        

if __name__ == '__main__':
    custom_YOLOv8 = Custom_YOLOv8(model_name='yolov8n.pt', filter=['person'])
    cap = cv2.VideoCapture(0)
    while True:
        # 웹캠 수신
        ret, img = cap.read()
        if ret == False:
            print('웹캠 수신 안됨')
            break
        
        # 더미 depth파일 생성
        h, w, c = img.shape
        depth_map = np.random.randint(0, 256, (w, h), dtype=np.uint8)

        # 인퍼런스
        ret, dic_list = custom_YOLOv8.run(img, depth_map)
        if ret:
            print(dic_list)
        draw_img = custom_YOLOv8.draw()
        
        cv2.imshow('test', draw_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cv2.destroyAllWindows()

자세한 기능 사용법 안내: https://github.com/Nyan-SouthKorea/YOLOv8_for_ROS2
GPU 사용 가능
[{'bbox': [66, 78, 624, 479], 'conf': 0.9, 'class_name': 'person', 'center_x': 345, 'center_y': 278, 'depth': 127.71}]
[{'bbox': [66, 79, 609, 479], 'conf': 0.9, 'class_name': 'person', 'center_x': 337, 'center_y': 279, 'depth': 127.22}]
[{'bbox': [68, 80, 606, 479], 'conf': 0.9, 'class_name': 'person', 'center_x': 337, 'center_y': 279, 'depth': 127.56}]
[{'bbox': [67, 79, 594, 479], 'conf': 0.9, 'class_name': 'person', 'center_x': 330, 'center_y': 279, 'depth': 127.05}]
[{'bbox': [67, 77, 598, 479], 'conf': 0.9, 'class_name': 'person', 'center_x': 332, 'center_y': 278, 'depth': 127.71}]
[{'bbox': [69, 77, 603, 479], 'conf': 0.9, 'class_name': 'person', 'center_x': 336, 'center_y': 278, 'depth': 127.24}]
[{'bbox': [69, 77, 603, 479], 'conf': 0.9, 'class_name': 'person', 'center_x': 336, 'center_y': 278, 'depth': 127.36}]
[{'bbox': [69, 77, 604, 479], 'conf': 0.89, 'class_name': 'person', 'center_x': 336, 'cente